#Odometer Detection - Stage 3

**Team Members**

Daysean Scott • Undergrad • (dscott52@uncc.edu) • 919-864-6832

Galal Chamma • Undergrad • (gchamma@uncc.edu) • 919-675-9673

Jaissruti Nanthakumar • Grad • (jnanthak@uncc.edu) • 336-405-4868

Janani Sri Adanur Rammohan • Grad • (jadanurr@uncc.edu) • 980-345-9218 


**GitLab:**
https://cci-git.uncc.edu/itcs-4152-5152/fall-2021/project-8-odometer



---



# Modeling
 
**Description of model**

The model takes the following steps to achieve the reading of the mileage:

1. Uses EasyOCR to find all text detections in the image

2. Determines whether a ‘mi’ or ‘km’ text is found.

3. Confirms that the text found is not ‘mph’ or ‘km/h’

4. Checks the largest length detections and confirms if a number is attached to it, and returns it.

5. If a number is not attached to the label text found, returns largest number found in image.


We created our own Class in order to store important information about our detections, such as the **top-left** and **bottom-right** coordinates of the bounding box, as well as the **detected text**. 
 

---



#Experimentation

1. How did you set up your experiments?

  By examining the provided Sonic Automotive images, we noticed a correlation between the position of the mileage number and its label, whether it was ‘mi’, ‘miles’, or ‘km’. That correlation allowed us to locate the mileage number after a series of steps that start with applying an optical character recognition algorithm over the image.
 
2. Datasets used: Train, Validation, Test split information

  Since our method, unlike a neural network, does not depend on training/testing split to work, we used approximately 20 images out of the provided 100 to come up with our solution. These 20 images had the mileage number in different locations compared to its label, and allowed us to come up with a solution that works with most of them.
 
3. Experimentation with different models or methods

  Besides this method, we are still experimenting with a neural network with training data composed of 2000+ images. So far, our current method has proved to work quite reliably with the images, but we will be checking whether a neural network would offer better results. 
 
4. Library and tools used

  EasyOCR, OpenCV, Matplotlib
 


---


#Literature Review
 
**Motivation for using method (cite any existing work that you have used)**
 
Effectiveness of Modern Text Recognition Solutions and Tools for Common Data Sources 

By: Kirill Smelyakov, Anastasiya Chupryna, Dmytro Darahan and Serhii Midina



http://ceur-ws.org/Vol-2870/paper15.pdf

* The motivation for using this method was sparked by the above mentioned paper, which describes the effectiveness and performance of two different OCR tools, EasyOCR and TesserOCR. The paper gave us the idea of detecting a key feature in the Odometer (the mileage label) using an OCR tool, and then determining the mileage based on that information.



---


#Results

1. **Model Performance**

  **Accuracy** = TP+TN / ALL | 98 + 0 / 113 = **87 %**

  **Precision** = TP / TP+FP  | 98 / 98 + 12 = **89 %**

  **Recall** = TP / TP+FN | 98 / 98 + 3 = **97 %**
 

2. **Visual Results**

  Using EasyOCR to determine the location of the bounding boxes and OpenCV to draw them, we were able to draw and label each detection as an overlay over the original image.
 
  We used this technique to examine all detections in an image, determine the best way to manipulate these detections to find the mileage number, and then only display the mileage number bounding box.
 
  Therefore, after running our algorithm, two output images are displayed. One containing all detections with their bounding boxes, in order to display everything that the OCR detected, and then a second image that only has the bounding box for the detection we used to extract the mileage number. Finally a print statement is displayed with the predicted mileage.
 
3. **Graphs**

  We do not have any graphs to show, since our method is not a neural network, therefore there are not any learning rate graphs, cost graphs, and so on. 
 
 

---


 
#Documentation
 
**A well presented notebook describing the above tasks** 

Please see our code below



In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
working_dir = '/content/drive/My Drive/Sonics Auto'
%cd '$working_dir'

/content/drive/My Drive/Sonics Auto


In [ ]:
import pandas as pd

df = pd.read_csv("labels.csv")

In [ ]:
true_mileage = []
for actual in df['odometer']:
  true_mileage.append(int(actual))


In [ ]:
!pip install easyocr

     |████████████████████████████████| 63.6 MB 1.3 MB/s 
     |████████████████████████████████| 47.6 MB 1.7 MB/s 


In [ ]:
import easyocr
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math


class Detection:
  def __init__(self, label, top_left, bottom_right):
    self.label = label
    self.top_left = top_left
    self.bottom_right = bottom_right

def cleanAndFind(validDetections, containsLabel):

  largestDetection = []
  maxFound = 0
  for det in validDetections:
    # If string contains a label such as 'mi' or 'km'
    if (containsLabel):
      # find the index of 'mi' in the string
      max_index = det.label.find('mi')  
      # if 'mi' was not found in the string, look for 'km' instead
      if (max_index == -1):
        max_index = det.label.find('km')
      # trimming the label from the number ('km...' or 'mi...')
      mileage = det.label[0:max_index]
    
    # text does not contain a label
    else:
      mileage = det.label

    # removing spaces in string
    mileage = mileage.replace(" ", "")

    nonDigit_index = 0
    for i, c in enumerate(mileage):
      if not c.isdigit():
          nonDigit_index = i
          break
    
    if (nonDigit_index != 0):
      mileage = mileage[0:nonDigit_index]

    if (mileage.isdecimal()):
      if (int(mileage) > maxFound):
        maxFound = int(mileage)
        largestDetection = [det,int(mileage)]

  return largestDetection


def displayImages(outputImages):
  plt.figure(figsize=(20,20)) # specifying the overall grid size

  for i in range(len(outputImages)):
    plt.subplot(1,2,i+1)    # the number of expected images in the grid is (1 row, 2 columns = 2)
    plt.imshow(outputImages[i])

  plt.show()

def drawBox(IMAGE_PATH, detection, mileage, outputImages):
    font = cv2.FONT_HERSHEY_SIMPLEX
    img = cv2.imread(IMAGE_PATH)
    img = cv2.rectangle(img,detection.top_left,detection.bottom_right,(0,255,0),3)
    img = cv2.putText(img,mileage,detection.top_left, font, 3,(255,255,255),7,cv2.LINE_AA)
    outputImages.append(img)

def find(image_path):

  outputImages = []
  IMAGE_PATH = image_path

  reader = easyocr.Reader(['en'])
  result = reader.readtext(IMAGE_PATH)
  img = cv2.imread(IMAGE_PATH)
  font = cv2.FONT_HERSHEY_SIMPLEX


  # Array to hold Detection objects, which have (Label, Top_Left, and Bottom_Right)
  detections = []

  for i in range (len(result)):
    # Fetching the data we care about from the result
    top_left = (int(result[i][0][0][0]),int(result[i][0][0][1]))
    bottom_right = (int(result[i][0][2][0]),int(result[i][0][2][1]))
    text = str(result[i][1])

    # creating an object for the label and information and adding them to array, i == ID
    det = Detection(text, top_left, bottom_right)
    detections.append(det)

    # Drawing the boxes and lables on the image
    img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
    img = cv2.putText(img,text,top_left, font, 2,(255,255,255),7,cv2.LINE_AA)

  
  outputImages.append(img)


  possibleDetections = []
  for det in detections:
    label = det.label
    # changed mi to m
    if ('mi' in label.lower() or 'km' in label.lower()):
      if (not ('km/' in label.lower() or 'mi/' in label.lower())):
        possibleDetections.append(det)

  # did we detect any 'mi' or 'km' ???
  if (len(possibleDetections)) > 0:

    # find the highest number WITH a label
    validDetection = cleanAndFind(possibleDetections, True)
    if (len(validDetection)) > 0:
      mileage = str(validDetection[1])
      drawBox(IMAGE_PATH, validDetection[0], mileage, outputImages)
      displayImages(outputImages)
      print("mileage detected:", mileage); return mileage
    
    # Cleaned array returned no mileage, but a label 'mi' or 'km' was actually found. return maximum number detected 
    else:
      #print("---- STEP 2 ACTIVATED ----")
      # using False for boolean since we don't care about slicing 'mi' or 'km' from strings
      validDetection = cleanAndFind(detections, False)
      if (len(validDetection)) > 0:
        mileage = str(validDetection[1])
        drawBox(IMAGE_PATH, validDetection[0], mileage, outputImages)
        displayImages(outputImages)
        print("mileage detected:", mileage); return mileage
      else:
        displayImages(outputImages)
        print("Mileage could not be detected "); return "NotFound"


  # no label was found in the image, return maximum number detected  
  else:
      # using False for boolean since we don't care about slicing 'mi' or 'km' from strings
      validDetection = cleanAndFind(detections, False)
      if (len(validDetection)) > 0:
        mileage = str(validDetection[1])
        drawBox(IMAGE_PATH, validDetection[0], mileage, outputImages)
        displayImages(outputImages)
        print("mileage detected:", mileage); return mileage
      else:
        displayImages(outputImages)
        print("Mileage could not be detected "); return "NotFound"

In [ ]:
predictions = []

counter = 1
for i, image in enumerate(df['filename']):
  print("\n\n")
  print("Currently examining image ["+ str(counter) + '/' + str(len(df))+"]")
  p = find('images/'+image)
  predictions.append(p)
  counter = counter+1
  if (p.isdecimal()):
    if (int(true_mileage[i]) == int(p)):
      print("Correct!")
    else:
      print("Incorrect :(")
  else:
    print("Incorrect :(")


for i in range(len(true_mileage)):
  print(i, ") ", "Actual: ", true_mileage[i], "      Prediction: ", predictions[i])

In [ ]:
tp = 0
fp = 0
fn = 0
trueFalse = []
for i in range(len(true_mileage)):
  if (str(predictions[i]) == "NotFound"):
    fn = fn + 1
    trueFalse.append("FALSE")
  if (str(true_mileage[i]) == str(predictions[i])):
    tp = tp + 1
    trueFalse.append("TRUE")
  elif (str(true_mileage[i]) != str(predictions[i]) and (str(predictions[i]) != "NotFound")):
    fp = fp + 1
    trueFalse.append("FALSE") 

print("All Data: ", len(true_mileage), 'images')
print("TP:", tp)
print("TN: 0")
print("FP:", fp)
print("FN:", fn)
print("-----------------")
print("Accuracy = TP+TN / ALL |", tp, "+ 0 /",len(true_mileage), "|", (tp/len(true_mileage)))
print("Recall = TP / TP+FN |", tp, "/",tp, "+", fn, "|", tp/(tp+fn))
print("Precision = TP / TP+FP  |", tp, "/",tp, "+", fp, "|", tp/(tp+fp))

All Data:  113 images
TP: 98
TN: 0
FP: 12
FN: 3
-----------------
Accuracy = TP+TN / ALL | 98 + 0 / 113 | 0.8672566371681416
Recall = TP / TP+FN | 98 / 98 + 3 | 0.9702970297029703
Precision = TP / TP+FP  | 98 / 98 + 12 | 0.8909090909090909


In [ ]:
# Saving the data to a csv file
df['Prediction'] = predictions
df['Validity'] = trueFalse
df1 = df.drop(columns=['xmin', 'xmax', 'ymin', 'ymax'])
df1.to_csv("prediction.csv", sep=",", encoding='utf-8', index=False)